In [1]:
import numpy as np
import pandas as pd

In [113]:
def train_elos(data, K=32):
    elos = {}
    for team in set(games_train[0]).union(set(games_train[2])):
        elos[team] = 1200.0
    
    for i in xrange(len(data)):
        game = data.iloc[i,]
        team1 = game[0]
        rating1 = elos[team1]
        team2 = game[2]
        rating2 = elos[team2]
        t_rating1 = 10**(rating1/400.0)
        t_rating2 = 10**(rating2/400.0)
        e_score1 = t_rating1/(t_rating1+t_rating2)
        e_score2 = t_rating2/(t_rating1+t_rating2)
        if game[1] > game[3]:
            s1 = 1
            s2 = 0
        else:
            s1 = 0
            s2 = 1
        u_rating1 = rating1 + K*(s1-e_score1)
        u_rating2 = rating2 + K*(s2-e_score2)
        elos[team1] = u_rating1
        elos[team2] = u_rating2
    
    return elos

def dump_elos(elos):
    import operator
    sorted_elos = sorted(elos.items(), key=operator.itemgetter(1))
    print sorted_elos[::-1]

In [142]:
def test_elos(model, data):
    data["TrueWinner"] = ""
    data["TrueLoser"] = ""
    data["PredWinner"] = ""
    data["PredLoser"] = ""
    for i in xrange(len(data)):
        game = data.iloc[i,]
        if game[1] > game[3]:
            data.iloc[i,5] = game[0]
            data.iloc[i,6] = game[2]
        else:
            data.iloc[i,5] = game[2]
            data.iloc[i,6] = game[0]
        if model[game[0]] > model[game[2]]:
            data.iloc[i,7] = game[0]
            data.iloc[i,8] = game[2]
        else:
            data.iloc[i,7] = game[2]
            data.iloc[i,8] = game[0]
    return data

In [144]:
games_train = pd.read_csv('games_train.csv', header=None)
elos = train_elos(games_train)
dump_elos(elos)
games_test = pd.read_csv('games_test.csv', header=None)
pred = test_elos(elos, games_test)

[('Golden State Warriors', 1476.5258348195412), ('San Antonio Spurs', 1423.2238663881353), ('Los Angeles Clippers', 1406.136960150845), ('Houston Rockets', 1373.1607606635648), ('Cleveland Cavaliers', 1351.9223062434462), ('Memphis Grizzlies', 1319.9456492661743), ('Atlanta Hawks', 1303.4547455427924), ('Charlotte Bobcats', 1291.061150991954), ('Dallas Mavericks', 1288.3396675567838), ('Boston Celtics', 1285.438775126209), ('New Orleans Pelicans', 1277.947289905757), ('Chicago Bulls', 1275.8417450600987), ('Oklahoma City Thunder', 1272.0080027005245), ('Utah Jazz', 1243.3131684966095), ('Portland Trail Blazers', 1240.0871708510501), ('Indiana Pacers', 1232.3964118278675), ('Toronto Raptors', 1222.439769224653), ('Brooklyn Nets', 1222.0668776984035), ('Washington Wizards', 1210.8854387603506), ('Milwaukee Bucks', 1170.8339470466851), ('Phoenix Suns', 1159.0838678331897), ('Miami Heat', 1152.862613944394), ('Detroit Pistons', 1135.5067491273198), ('Denver Nuggets', 1106.658694930612), ('

In [152]:
(sum(pred["TrueWinner"] == pred["PredWinner"])+0.0)/len(pred)

0.64715447154471539

In [4]:
games = pd.read_csv('games.csv', header=None)

In [45]:
df1 = pd.DataFrame([games[0], games[1]-games[3], games[4]]).T

In [47]:
df2 = pd.DataFrame([games[2], games[3]-games[1], games[4]]).T

In [50]:
df1 = df1.rename(columns={0: 'Team', 'Unnamed 0': 'ScoreDiff', 4: 'Year'})
df2 = df2.rename(columns={2: 'Team', 'Unnamed 0': 'ScoreDiff', 4: 'Year'})

In [51]:
team_scores = pd.concat([df1, df2], ignore_index = True)

In [90]:
elos = {}
for team in set(team_scores['Team']):
    elos[team] = 1200.0
    
K = 32
for i in xrange(len(games)):
    game = games.iloc[i,]
    team1 = game[0]
    rating1 = elos[team1]
    team2 = game[2]
    rating2 = elos[team2]
    t_rating1 = 10**(rating1/400.0)
    t_rating2 = 10**(rating2/400.0)
    e_score1 = t_rating1/(t_rating1+t_rating2)
    e_score2 = t_rating2/(t_rating1+t_rating2)
    if game[1] > game[3]:
        s1 = 1
        s2 = 0
    else:
        s1 = 0
        s2 = 1
    u_rating1 = rating1 + K*(s1-e_score1)
    u_rating2 = rating2 + K*(s2-e_score2)
    elos[team1] = u_rating1
    elos[team2] = u_rating2

In [91]:
import operator
sorted_elos = sorted(elos.items(), key=operator.itemgetter(1))

In [92]:
sorted_elos[::-1]

[('Golden State Warriors', 1456.2734122018471),
 ('San Antonio Spurs', 1420.0698279396202),
 ('Los Angeles Clippers', 1334.6113332468972),
 ('Oklahoma City Thunder', 1319.9166908308803),
 ('Toronto Raptors', 1297.5066622781537),
 ('Cleveland Cavaliers', 1292.916946192631),
 ('Atlanta Hawks', 1275.2306813106652),
 ('Portland Trail Blazers', 1265.0847217165378),
 ('Houston Rockets', 1262.002588542613),
 ('Memphis Grizzlies', 1251.1797294712494),
 ('Miami Heat', 1248.6807166764),
 ('Dallas Mavericks', 1247.3391242182256),
 ('Indiana Pacers', 1232.7854715665717),
 ('Boston Celtics', 1230.3487170988403),
 ('Chicago Bulls', 1227.0336387282189),
 ('Charlotte Hornets', 1223.1701585817657),
 ('Washington Wizards', 1206.1279348975627),
 ('Utah Jazz', 1194.1436246952076),
 ('Detroit Pistons', 1186.2393903562188),
 ('New Orleans Pelicans', 1158.7471597670983),
 ('Charlotte Bobcats', 1152.5008488608917),
 ('Denver Nuggets', 1148.8560414712506),
 ('New Jersey Nets', 1135.5870942840347),
 ('Milwaukee